# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using scikit-learn). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up pandas, numpy and sklearn

In [4]:
import pandas as pd
import numpy as np
from sklearn import datasets, linear_model
from math import log

# Load in house sales data

In [6]:
# Load all dataset
sales = pd.read_csv('kc_house_data.csv')

# Create new features

Create new features by performing following transformation on inputs:

In [8]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']
sales['floors_square'] = sales['floors']*sales['floors']

In [10]:
sales['floors_square'].dtype

dtype('float64')

*Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.

*On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

2.. Using the entire house dataset, learn regression weights using an L1 penalty of 5e2. Make sure to add "normalize=True" when creating the Lasso object. Refer to the following code snippet for the list of features.

Note. From here on, the list 'all_features' refers to the list defined in this snippet.

In [11]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

model_all = linear_model.Lasso(alpha=5e2, normalize=True) # set parameters
model_all.fit(sales[all_features], sales['price']) # learn weights

Lasso(alpha=500.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=True, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

3.. Quiz Question: Which features have been chosen by LASSO, i.e. which features were assigned nonzero weights?

In [12]:
model_all.coef_

array([     0.        ,      0.        ,      0.        ,    134.43931396,
            0.        ,      0.        ,      0.        ,      0.        ,
            0.        ,      0.        ,  24750.00458561,      0.        ,
        61749.10309071,      0.        ,      0.        ,     -0.        ,
            0.        ])

Quiz Answer: 'sqft_living', 'view', 'grade'


In [13]:
testing = pd.read_csv('wk3_kc_house_test_data.csv')
training = pd.read_csv('wk3_kc_house_train_data.csv')
validation = pd.read_csv('wk3_kc_house_valid_data.csv')

Make sure to create the 4 features as we did in #1:

In [14]:
testing['sqft_living_sqrt'] = testing['sqft_living'].apply(sqrt)
testing['sqft_lot_sqrt'] = testing['sqft_lot'].apply(sqrt)
testing['bedrooms_square'] = testing['bedrooms']*testing['bedrooms']
testing['floors_square'] = testing['floors']*testing['floors']

training['sqft_living_sqrt'] = training['sqft_living'].apply(sqrt)
training['sqft_lot_sqrt'] = training['sqft_lot'].apply(sqrt)
training['bedrooms_square'] = training['bedrooms']*training['bedrooms']
training['floors_square'] = training['floors']*training['floors']

validation['sqft_living_sqrt'] = validation['sqft_living'].apply(sqrt)
validation['sqft_lot_sqrt'] = validation['sqft_lot'].apply(sqrt)
validation['bedrooms_square'] = validation['bedrooms']*validation['bedrooms']
validation['floors_square'] = validation['floors']*validation['floors']

5.. Now for each l1_penalty in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type np.logspace(1, 7, num=13).)

~Learn a model on TRAINING data using the specified l1_penalty. Make sure to specify normalize=True in the constructor:

~Compute the RSS on VALIDATION for the current model (print or save the RSS)

~Report which L1 penalty produced the lower RSS on VALIDATION.

In [26]:
RSS = []
for l1_penalty in np.logspace(1, 7, num=13):
    # Learn a model on TRAINING data using the specified l1_penalty.
    model = linear_model.Lasso(alpha=l1_penalty, normalize=True)
    model_1 = model.fit(training[all_features], training['price'])
    # Predit price based on model_1 and valication data
    valid_prediction = model_1.predict(validation[all_features])
    # Calculate erros
    valid_errors = valid_prediction - validation['price']
    # Calculate RSS
    valid_RSS = (valid_errors * valid_errors).sum()
    RSS.append(valid_RSS)
    print(valid_RSS)
    print(l1_penalty)
min(RSS)

3.982133273e+14
10.0
3.99041900253e+14
31.6227766017
4.29791604073e+14
100.0
4.63739831045e+14
316.227766017
6.45898733634e+14
1000.0
1.22250685943e+15
3162.27766017
1.22250685943e+15
10000.0
1.22250685943e+15
31622.7766017
1.22250685943e+15
100000.0
1.22250685943e+15
316227.766017
1.22250685943e+15
1000000.0
1.22250685943e+15
3162277.66017
1.22250685943e+15
10000000.0


398213327300134.94

6.. Quiz Question: Which was the best value for the l1_penalty, i.e. which value of l1_penalty produced the lowest RSS on VALIDATION data?

Answer: 10

7.. Now that you have selected an L1 penalty, compute the RSS on TEST data for the model with the best L1 penalty.

In [28]:
l1_penalty = 10
model = linear_model.Lasso(alpha=l1_penalty, normalize=True)
model_2 = model.fit(training[all_features], training['price'])
# Predit price based on model_1 and valication data
test_prediction = model_2.predict(testing[all_features])
# Calculate erros
test_errors = test_prediction - testing['price']
# Calculate RSS
test_RSS = (test_errors * test_errors).sum()
test_RSS

98467402552698.797

8.. Quiz Question: Using the best L1 penalty, how many nonzero weights do you have? Count the number of nonzero coefficients first, and add 1 if the intercept is also nonzero. A succinct way to do this is

In [29]:
np.count_nonzero(model_2.coef_) + np.count_nonzero(model_2.intercept_)

15

# Limit the number of nonzero weights

9.. What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

You are going to implement a simple, two phase procedure to achieve this goal:
    
~Explore a large range of ‘l1_penalty’ values to find a narrow region of ‘l1_penalty’ values where models are likely to have the desired number of non-zero weights.

~Further explore the narrow region you found to find a good value for ‘l1_penalty’ that achieves the desired sparsity. Here, we will again use a validation set to choose the best value for ‘l1_penalty’.

10.. Assign 7 to the variable ‘max_nonzeros’.

In [30]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

11.. Exploring large range of l1_penalty

For l1_penalty in np.logspace(1, 4, num=20):

Fit a regression model with a given l1_penalty on TRAIN data. Add "alpha=l1_penalty" and "normalize=True" to the parameter list.

Extract the weights of the model and count the number of nonzeros. Take account of the intercept as we did in #8, adding 1 whenever the intercept is nonzero. Save the number of nonzeros to a list.

In [32]:
num = []
for l1_penalty in np.logspace(1, 4, num=20):
    # Learn a model on TRAINING data using the specified l1_penalty.
    model = linear_model.Lasso(alpha=l1_penalty, normalize=True)
    model_3 = model.fit(training[all_features], training['price'])
    nonzeros = np.count_nonzero(model_3.coef_) + np.count_nonzero(model_3.intercept_)
    num.append(nonzeros)
num

[15, 15, 15, 15, 13, 12, 11, 10, 7, 6, 6, 6, 5, 3, 3, 2, 1, 1, 1, 1]

In [33]:
np.logspace(1, 4, num=20)

array([    10.        ,     14.38449888,     20.69138081,     29.76351442,
           42.81332399,     61.58482111,     88.58667904,    127.42749857,
          183.29807108,    263.66508987,    379.26901907,    545.55947812,
          784.75997035,   1128.83789168,   1623.77673919,   2335.72146909,
         3359.81828628,   4832.93023857,   6951.92796178,  10000.        ])

12.. Out of this large range, we want to find the two ends of our desired narrow range of l1_penalty. At one end, we will have l1_penalty values that have too few non-zeros, and at the other end, we will have an l1_penalty that has too many non-zeros.

More formally, find:

The largest l1_penalty that has more non-zeros than ‘max_nonzeros’ (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)Store this value in the variable ‘l1_penalty_min’ (we will use it later)

The smallest l1_penalty that has fewer non-zeros than ‘max_nonzeros’ (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)Store this value in the variable ‘l1_penalty_max’ (we will use it later)

Hint: there are many ways to do this, e.g.:

Programmatically within the loop above
Creating a list with the number of non-zeros for each value of l1_penalty and inspecting it to find the appropriate boundaries.

Answer:
#the largest l1_penalty: l1_penalty_min = 127.42749857
#the smallest l1_penalty:  l1_penalty_max = 263.66508987

14.. Exploring narrower range of l1_penalty

We now explore the region of l1_penalty we found: between ‘l1_penalty_min’ and ‘l1_penalty_max’. We look for the L1 penalty in this range that produces exactly the right number of nonzeros and also minimizes RSS on the VALIDATION set.

For l1_penalty in np.linspace(l1_penalty_min,l1_penalty_max,20):

Fit a regression model with a given l1_penalty on TRAIN data. As before, use "alpha=l1_penalty" and "normalize=True".
Measure the RSS of the learned model on the VALIDATION set
Find the model that the lowest RSS on the VALIDATION set and has sparsity equal to ‘max_nonzeros’. (Again, take account of the intercept when counting the number of nonzeros.)

In [44]:
l1_penalty_min = 127.42749857
l1_penalty_max = 263.66508987
sparsity = []
Valid_RSS = []
l1_penalty_list =[]
for l1_penalty in np.linspace(l1_penalty_min,l1_penalty_max,20):
    #print(l1_penalty)
    l1_penalty_list.append(l1_penalty)
    # Learn a model on TRAINING data using the specified l1_penalty.
    model = linear_model.Lasso(alpha=l1_penalty, normalize=True)
    model_4 = model.fit(training[all_features], training['price'])
    nonzeros = np.count_nonzero(model_4.coef_) + np.count_nonzero(model_4.intercept_)
    sparsity.append(nonzeros)
    # Predit price based on model_4 and validation data
    valid_prediction = model_4.predict(validation[all_features])
    # Calculate erros
    valid_errors = valid_prediction - validation['price']
    # Calculate RSS
    RSS_v = (valid_errors * valid_errors).sum()
    Valid_RSS.append(RSS_v)
print(l1_penalty_list)
print(sparsity)
print(Valid_RSS)

[127.42749857, 134.59789811210527, 141.76829765421053, 148.93869719631579, 156.10909673842104, 163.2794962805263, 170.44989582263156, 177.62029536473682, 184.79069490684208, 191.96109444894734, 199.13149399105259, 206.30189353315785, 213.47229307526311, 220.64269261736837, 227.81309215947365, 234.98349170157891, 242.15389124368417, 249.32429078578943, 256.49469032789472, 263.66508986999997]
[10, 10, 8, 8, 7, 7, 7, 7, 7, 7, 7, 6, 6, 6, 6, 6, 6, 6, 6, 6]
[435374677102611.62, 437009229124363.88, 438236128386837.94, 439158937799561.0, 440037365263228.5, 440777489641495.5, 441566698090006.62, 442406413188507.38, 443296716874128.62, 444239780525925.75, 445230739842366.81, 446268896864489.31, 447112919434395.5, 447998187851290.0, 448924706672948.69, 449892475899371.5, 450901498777749.0, 451952426654576.5, 453043924367150.56, 454176669662146.88]


In [48]:
df = pd.DataFrame({'l1_penalty':l1_penalty_list,'sparsity':sparsity,'Valid_RSS':Valid_RSS})
df

,l1_penalty,sparsity,Valid_RSS
0,127.427499,10,4.353747e+14
1,134.597898,10,4.370092e+14
2,141.768298,8,4.382361e+14
3,148.938697,8,4.391589e+14
4,156.109097,7,4.400374e+14
5,163.279496,7,4.407775e+14
6,170.449896,7,4.415667e+14
7,177.620295,7,4.424064e+14
8,184.790695,7,4.432967e+14
9,191.961094,7,4.442398e+14


In [51]:
df.sort_values(by=['Valid_RSS'])

,l1_penalty,sparsity,Valid_RSS
0,127.427499,10,4.353747e+14
1,134.597898,10,4.370092e+14
2,141.768298,8,4.382361e+14
3,148.938697,8,4.391589e+14
4,156.109097,7,4.400374e+14
5,163.279496,7,4.407775e+14
6,170.449896,7,4.415667e+14
7,177.620295,7,4.424064e+14
8,184.790695,7,4.432967e+14
9,191.961094,7,4.442398e+14


15.. Quiz Question: What value of l1_penalty in our narrow range has the lowest RSS on the VALIDATION set and has sparsity equal to ‘max_nonzeros’?

Answer: 156.109097

16.. Quiz Question: What features in this model have non-zero coefficients?

In [54]:
l1_penalty = 156.109097
# Learn a model on TRAINING data using the specified l1_penalty.
model = linear_model.Lasso(alpha=l1_penalty, normalize=True)
model_5 = model.fit(training[all_features], training['price'])
model_5.coef_

array([ -0.00000000e+00,  -0.00000000e+00,   1.06108902e+04,
         1.63380252e+02,   0.00000000e+00,  -0.00000000e+00,
        -0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         5.06451687e+05,   4.19600436e+04,   0.00000000e+00,
         1.16253554e+05,   0.00000000e+00,   0.00000000e+00,
        -2.61223488e+03,   0.00000000e+00])

Answer: feature 3, 4, 10,11,13,16

In [56]:
all_features

['bedrooms',
 'bedrooms_square',
 'bathrooms',
 'sqft_living',
 'sqft_living_sqrt',
 'sqft_lot',
 'sqft_lot_sqrt',
 'floors',
 'floors_square',
 'waterfront',
 'view',
 'condition',
 'grade',
 'sqft_above',
 'sqft_basement',
 'yr_built',
 'yr_renovated']

Answer:'bathrooms','sqft_living','waterfront','view','grade','yr_built'